# Quevo-1 - Example project

This notebook shows how you may use the Quevo to find quantum circuits fittet to a 1D Von Neumann neigbourhood for cellular automata. This guide will go through the process of initialising the needed parameters, creating chromosomes generations and in the end simulationg them on the qiskit simulator to see the quantum results.

First we import the Quevo library. __Note!__ You need Qiskit for the library to work.

In [33]:
import numpy as np
from Quevo import *
from qiskit.visualization import circuit_drawer

Then we set up the evovlers parameters. 
- **gates**: The number of quantum gates per circuit.
- **chromosomes**: The number of chromosomes or solutions you want in each generation.
- **generation**: The number of generations you want to evolve before terminating.
- **desired_chance_of_one**: A list of the eight probabilities for the eight CA initial states you want the circuit to generate. There are some example probability list commented. Remove the comment to test them!

In [34]:
gates = 10
n_states = 8 
chromosomes = 10
generations = 20
gate_types = ['cx', 'x', 'h', 'rxx', 'rzz', 'swap', 'z', 'y', 'toffoli']  # possible gates: # h, cx, x, y, z, swap, rzz, rxx, toffoli
states = np.random.rand(n_states, 2**3)

In [35]:
# Generate initial generation of chromosomes
init_gen = Generation(chromosomes, gates)
init_gen.create_initial_generation(gate_types)
init_gen.print_chromosomes()



Chromosomes: 
[1, 2, 0, 1, 1, 1, 8, 0, 1, 0, 2, 0, 1, 2, 1, 6, 1, 2, 7, 0, 2, 8, 1, 1, 1, 2, 0, 6, 1, 2]
[1, 1, 0, 4, 0, 1, 3, 2, 0, 0, 2, 0, 5, 2, 0, 1, 0, 2, 5, 2, 0, 3, 1, 0, 4, 2, 1, 3, 2, 1]
[6, 0, 2, 7, 1, 2, 8, 2, 1, 0, 2, 0, 3, 0, 2, 0, 2, 1, 2, 2, 2, 7, 1, 2, 2, 1, 0, 1, 2, 2]
[0, 2, 0, 1, 1, 0, 8, 2, 0, 2, 0, 0, 4, 0, 2, 4, 0, 2, 3, 1, 0, 5, 0, 1, 1, 0, 1, 0, 1, 0]
[2, 0, 0, 6, 2, 0, 4, 1, 0, 6, 2, 1, 8, 2, 1, 7, 1, 1, 7, 0, 0, 1, 0, 1, 0, 1, 0, 3, 2, 0]
[7, 2, 1, 2, 1, 0, 5, 2, 0, 3, 0, 1, 0, 1, 0, 4, 0, 2, 6, 0, 2, 3, 1, 2, 3, 1, 0, 7, 1, 0]
[8, 1, 0, 4, 2, 1, 6, 2, 1, 2, 1, 1, 8, 0, 1, 4, 0, 1, 3, 2, 0, 6, 1, 1, 8, 2, 0, 8, 0, 0]
[8, 0, 0, 7, 1, 1, 7, 1, 1, 4, 1, 0, 1, 2, 0, 3, 2, 1, 6, 0, 0, 4, 0, 2, 0, 0, 2, 2, 2, 0]
[0, 1, 0, 7, 1, 2, 6, 2, 0, 8, 1, 0, 6, 2, 0, 7, 1, 2, 1, 0, 0, 0, 0, 2, 4, 0, 2, 7, 2, 2]
[5, 0, 2, 2, 1, 2, 5, 1, 2, 3, 0, 2, 4, 1, 0, 0, 2, 1, 4, 0, 2, 8, 0, 1, 5, 1, 2, 8, 1, 2]




Then we check all the chromosomes fitness and print the best one found.

In [36]:
init_gen.run_generation(states)

print("Fitness for best chromosome: " + str(init_gen.get_best_fitness()) + "\n"
      + "Best generated circuit: \n" + str(init_gen.get_best_chromosome()))
print("\n")

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

MW_entanglement = 0.33883299342150397

Fitness for best chromosome: 0.33883299342150397
Best generated circuit: 
[1, 2, 0, 1, 1, 1, 8, 0, 1, 0, 2, 0, 1, 2, 1, 6, 1, 2, 7, 0, 2, 8, 1, 1, 1, 2, 0, 6, 1, 2]




Before we can start the evolution, we need to declare some container values fopr the best performing chromosome.

In [37]:
# Final value placeholders
current_chromosome = init_gen.get_best_chromosome()
best_chromosome = current_chromosome
final_fitness = init_gen.get_best_fitness()

Then we create a loop for the number of generations, and every time find the best chromosome in the generation to be a parent for the next generation.

In [38]:
# Mutation loop
for gen in range(0, generations):

    # Mutate next generation of chromosomes
    next_gen = Generation(chromosomes, gates)
    next_gen.create_mutated_generation(current_chromosome)

    # Check every Chromosome's fitness
    next_gen.run_generation(states)

    current_fitness = next_gen.get_best_fitness()
    current_chromosome = next_gen.get_best_chromosome()
    
    # Print generation best result
    print("Fitness for best mutated chromosome in mutation " + str(gen + 1) + ": "
          + str(current_fitness) + "\n"
          + "Best mutated chromosome:\n" + str(next_gen.get_best_chromosome()))
    print("\n")

    # Check if there is a new_list best chromosome

    if final_fitness > current_fitness:
        final_fitness = current_fitness
        best_chromosome = current_chromosome
        print("New best!")
        
        print("\n")
    if current_fitness < 0.01: # Stop if high enough fitness is reached
        break
    print("------------------------------------------------------------------------------")


AttributeError: 'Generation' object has no attribute 'create_mutated_generation'

Last, let's print the result of the best found ciurcuit and visualize it.

In [42]:
print("Best fitness found: " + str(final_fitness))
print("Best chromosome found: " + str(best_chromosome))
print("\n")
circuit = Circuit(best_chromosome)



circuit.generate_circuit()
circuit.draw()


Best fitness found: 0.33883299342150397
Best chromosome found: [1, 2, 0, 1, 1, 1, 8, 0, 1, 0, 2, 0, 1, 2, 1, 6, 1, 2, 7, 0, 2, 8, 1, 1, 1, 2, 0, 6, 1, 2]


          ┌───┐┌───┐┌───┐          ┌─┐
q_0: ─────┤ X ├┤ X ├┤ Y ├──■───────┤M├
     ┌───┐└─┬─┘└─┬─┘├───┤┌─┴─┐┌───┐└╥┘
q_1: ┤ X ├──■────┼──┤ Z ├┤ X ├┤ Z ├─╫─
     ├───┤  │    │  ├───┤└─┬─┘├───┤ ║ 
q_2: ┤ X ├──■────■──┤ X ├──■──┤ X ├─╫─
     └───┘          └───┘     └───┘ ║ 
c: 1/═══════════════════════════════╩═
                                    0 
